In [1]:
print("hello there")

hello there


In [2]:
import carla
import numpy as np
import cv2
import threading
import queue
import pygame
import time
from ultralytics import YOLO

# Connect to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

# Constants for safety systems
ABS_ACTIVATION_PCT = 35
FULL_STOP_PCT = 45
CENTER_REGION = 0.2
MIN_DISTANCE = 20
FOCAL_LENGTH = 320
VEHICLE_WIDTH = 2.0
CROSSWALK_TARGET_SPEED = 20
STOP_TARGET_SPEED = 0
BASE_BRAKE_FACTOR = 0.3
ABS_COOLDOWN = 5
STOP_SIGN_HOLD_TIME = 3

# Get blueprint library and spawn points
blueprint_library = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()
if not spawn_points:
    print("No spawn points available. Exiting...")
    exit()

# Spawn the vehicle
vehicle_bp = blueprint_library.filter('vehicle.dodge.charger')[0]
vehicle = world.spawn_actor(vehicle_bp, spawn_points[0])

# Attach an RGB camera (CORRECTED CAMERA SETUP)
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute("image_size_x", "640")
camera_bp.set_attribute("image_size_y", "480")
camera_bp.set_attribute("fov", "90")
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.0))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

# Load YOLO models
vehicle_model = YOLO("vehicle_detector(y11).pt")
road_model = YOLO("road_marking_detector.pt")

# Shared variables for safety systems
max_vehicle_area = 0.0
vehicle_distance = 100.0
vehicle_area_rate = 0.0
reverse_gear = False
prev_area = 0.0
prev_time = time.time()
crosswalk_detected = False
stop_sign_detected = False
red_light_detected = False
current_speed = 0.0
last_abs_activation_time = 0
last_stop_time = 0
data_lock = threading.Lock()

# Frame queue
frame_queue = queue.Queue(maxsize=2)

def calculate_distance(box_width):
    return (VEHICLE_WIDTH * FOCAL_LENGTH) / box_width

def get_current_speed():
    velocity = vehicle.get_velocity()
    return 3.6 * (velocity.x**2 + velocity.y**2 + velocity.z**2)**0.5

def image_processing_worker():
    global max_vehicle_area, prev_area, prev_time, vehicle_area_rate, vehicle_distance
    global crosswalk_detected, stop_sign_detected, red_light_detected
    while True:
        try:
            frame = frame_queue.get(timeout=1)
            while not frame_queue.empty():
                frame = frame_queue.get()
            array = np.frombuffer(frame.raw_data, dtype=np.uint8)
            array = array.reshape((frame.height, frame.width, 4))[:, :, :3]
            writable_array = array.copy()

            height, width = 480, 640
            center_left = int(width * (0.5 - CENTER_REGION/2))
            center_right = int(width * (0.5 + CENTER_REGION/2))
            
            cv2.rectangle(writable_array, (center_left, 0), (center_right, height), (0, 165, 255), 2)

            # Vehicle detection
            vehicle_results = vehicle_model(writable_array, verbose=False)
            current_max_area = 0.0
            current_distance = 100.0
            total_pixels = width * height
            
            for result in vehicle_results:
                for box in result.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    class_id = int(box.cls[0])
                    class_name = vehicle_model.names[class_id]
                    
                    if class_name in ['car', 'truck', 'bus', 'motorcycle']:
                        box_center_x = (x1 + x2) // 2
                        in_center = center_left <= box_center_x <= center_right
                        
                        # Default color for vehicles outside the center
                        color = (0, 255, 0)  # Green
                        distance = None
                        
                        if in_center:
                            box_width = x2 - x1
                            distance = calculate_distance(box_width)
                            
                            # Update closest distance and max area
                            if distance < current_distance:
                                current_distance = distance
                                area = (x2 - x1) * (y2 - y1)
                                area_percent = (area / total_pixels) * 100
                                current_max_area = max(current_max_area, area_percent)
                            
                            # Set color based on distance for center vehicles
                            color = (255, 0, 0) if distance < MIN_DISTANCE else (255, 255, 0)
                        
                        # Draw bounding box for all vehicles
                        cv2.rectangle(writable_array, (x1, y1), (x2, y2), color, 2)
                        
                        # Add text based on position
                        if in_center and distance is not None:
                            cv2.putText(writable_array, f"{distance:.1f}m", (x1, y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                        else:
                            cv2.putText(writable_array, class_name, (x1, y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            current_time = time.time()
            time_diff = current_time - prev_time
            if time_diff > 0:
                area_diff = current_max_area - prev_area
                current_rate = (area_diff / time_diff) * 100
            else:
                current_rate = 0.0

            with data_lock:
                max_vehicle_area = current_max_area
                vehicle_area_rate = current_rate
                vehicle_distance = current_distance

            prev_area = current_max_area
            prev_time = current_time

            # Road object detection
            road_results = road_model(writable_array, verbose=False)
            crosswalk_temp = False
            stop_sign_temp = False
            red_light_temp = False
            
            for result in road_results:
                for box in result.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    confidence = box.conf[0]
                    class_id = int(box.cls[0])
                    class_name = road_model.names[class_id]
                    label = f"{class_name}: {confidence:.2f}"
                    
                    # Draw bounding boxes
                    color = (255, 0, 0)
                    if class_name == 'Crosswalk':
                        color = (0, 255, 255)
                        crosswalk_temp = True
                    elif class_name == 'Stop_Sign':
                        color = (0, 0, 255)
                        stop_sign_temp = True
                    elif class_name == 'Traffic_Light_Red':
                        color = (0, 0, 255)
                        red_light_temp = True
                    
                    cv2.rectangle(writable_array, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(writable_array, label, (x1, y2 + 15),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            with data_lock:
                crosswalk_detected = crosswalk_temp
                stop_sign_detected = stop_sign_temp
                red_light_detected = red_light_temp

            cv2.imshow("Smart Collision Prevention", writable_array)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        except queue.Empty:
            continue
    cv2.destroyAllWindows()

worker_thread = threading.Thread(target=image_processing_worker, daemon=True)
worker_thread.start()

def camera_callback(image):
    if not frame_queue.full():
        frame_queue.put(image)

camera.listen(camera_callback)

pygame.init()
screen = pygame.display.set_mode((400, 300))
pygame.display.set_caption("Manual Control (WASD/R Keys)")

# Control parameters
throttle = 0.0
steering_angle = 0.0
brake = 0.0
max_steering_angle = 1.0
stopped_at_stop = False

try:
    while True:
        current_speed = get_current_speed()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                cv2.destroyAllWindows()
                camera.destroy()
                vehicle.destroy()
                exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:
                    reverse_gear = not reverse_gear

        keys = pygame.key.get_pressed()

        if keys[pygame.K_w]:
            throttle = 1.0
            brake = 0.0
        elif keys[pygame.K_s]:
            throttle = 0.0
            brake = 1.0
        else:
            throttle = 0.0
            brake = 0.0

        if keys[pygame.K_a]:
            steering_angle = -max_steering_angle
        elif keys[pygame.K_d]:
            steering_angle = max_steering_angle
        else:
            steering_angle = 0.0

        # Safety systems
        auto_brake_active = False
        emergency_brake = False
        abs_brake = 0.0
        traffic_brake = 0.0
        
        if not reverse_gear:
            with data_lock:
                current_max = max_vehicle_area
                approach_rate = vehicle_area_rate
                distance = vehicle_distance
                stop_sign = stop_sign_detected
                red_light = red_light_detected
                crosswalk = crosswalk_detected

            # Collision prevention braking
            base_brake = 0.0
            if current_max >= ABS_ACTIVATION_PCT and distance < MIN_DISTANCE:
                base_brake = ((current_max - ABS_ACTIVATION_PCT) / 
                            (FULL_STOP_PCT - ABS_ACTIVATION_PCT))
                distance_factor = 1 - (distance / MIN_DISTANCE)
                base_brake = max(base_brake, distance_factor)
            
            rate_brake = min(approach_rate / 15, 1.0) if approach_rate > 0 else 0.0
            abs_brake = min(base_brake + rate_brake, 1.0)
            
            if distance < MIN_DISTANCE/2 or current_max >= FULL_STOP_PCT:
                abs_brake = 1.0
                emergency_brake = True

            current_time = time.time()
            if (current_max >= ABS_ACTIVATION_PCT or emergency_brake) and distance < MIN_DISTANCE:
                if current_speed < 5:
                    if (current_time - last_abs_activation_time) >= ABS_COOLDOWN:
                        auto_brake_active = True
                        last_abs_activation_time = current_time
                    else:
                        auto_brake_active = False
                        abs_brake = 0.0
                else:
                    auto_brake_active = True

                if auto_brake_active:
                    throttle = 0.0
            else:
                auto_brake_active = False

            # Traffic control braking
            if stop_sign or red_light or crosswalk:
                target_speed = STOP_TARGET_SPEED if (stop_sign or red_light) else CROSSWALK_TARGET_SPEED
                speed_diff = current_speed - target_speed
                
                if speed_diff > 0:
                    traffic_brake = min(BASE_BRAKE_FACTOR * (speed_diff / (target_speed + 1)), 0.8)
                    throttle *= 0.3
                else:
                    traffic_brake = 0.0
                    throttle = min(throttle * 1.2, 1.0) if not (stop_sign or red_light) else 0.0

                # Full stop logic
                if current_speed < 1 and (stop_sign or red_light):
                    traffic_brake = 0.8
                    throttle = 0.0
                    if not stopped_at_stop:
                        last_stop_time = current_time
                        stopped_at_stop = True
                    
                    if (current_time - last_stop_time) < STOP_SIGN_HOLD_TIME:
                        traffic_brake = 0.5
                else:
                    stopped_at_stop = False

        brake = max(brake, abs_brake, traffic_brake)

        # Vehicle control
        control = carla.VehicleControl(
            throttle=throttle,
            steer=steering_angle,
            brake=brake,
            reverse=reverse_gear
        )
        vehicle.apply_control(control)

        # Display
        screen.fill((0, 0, 0))
        font = pygame.font.Font(None, 36)
        
        speed_text = font.render(f"Speed: {current_speed:.1f} km/h", True, (255, 255, 255))
        screen.blit(speed_text, (10, 10))
        
        gear_color = (255, 0, 0) if reverse_gear else (0, 255, 0)
        gear_text = font.render("REVERSE" if reverse_gear else "DRIVE", True, gear_color)
        screen.blit(gear_text, (10, 50))
        
        status_y = 90
        if brake > 0:
            brake_text = font.render(f"Braking: {brake*100:.0f}%", True, (0, 0, 255))
            screen.blit(brake_text, (10, status_y))
            status_y += 40
            
            if abs_brake > 0:
                status_text = font.render("Collision Prevention", True, (255, 0, 0))
                screen.blit(status_text, (10, status_y))
                status_y += 40
                
            if traffic_brake > 0:
                status_color = (0, 255, 255)
                status_text = ""
                if crosswalk_detected:
                    status_text = "Crosswalk"
                if stop_sign_detected:
                    status_text = "Stop Sign"
                if red_light_detected:
                    status_text = "Red Light"
                if status_text:
                    status_text += " Speed Control"
                    status_render = font.render(status_text, True, status_color)
                    screen.blit(status_render, (10, status_y))
                    status_y += 40
                
        if reverse_gear:
            status_text = font.render("ABS Disabled", True, (255, 165, 0))
            screen.blit(status_text, (10, status_y))

        pygame.display.flip()
        world.tick()

except KeyboardInterrupt:
    print("Simulation stopped manually")

finally:
    camera.destroy()
    vehicle.destroy()
    cv2.destroyAllWindows()
    pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.